In [1]:
import nltk
import pickle
import pandas as pd
import numpy as np
import gensim
import re  # regular expressions (for playing with the text)
from gensim.parsing.preprocessing import STOPWORDS # common english "stop words" -- a, the, etc.
from gensim.parsing import PorterStemmer
from gensim import corpora, models
import nltk.data
import emoji
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from csv import reader
import csv
from nltk.corpus import stopwords
from string import punctuation
import csv
from nltk import bigrams 
from nltk import trigrams
import datetime, pytz, time
import os, json
import pandas as pd
from collections import Counter
from pytz import timezone
from datetime import datetime
from dateutil import tz
import string
import html
import pandas as pd
import csv
import re
import codecs
import unicodedata
import string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize
from textblob import TextBlob
sid = SentimentIntensityAnalyzer()
path_to_json = 'reddit_data/'
import time
from datetime import timedelta

In [2]:
import subprocess
 
def run_cmd(args_list):
        print('Running system command: {0}'.format(' '.join(args_list)))
        proc = subprocess.Popen(args_list, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        s_output, s_err = proc.communicate()
        s_return =  proc.returncode
        return s_return, s_output, s_err 

def apos_words(text):
    sentence=[]
    for word in text.split():
        try:
            sentence.append(apos_dict[word.lower()])
        except:
            sentence.append(' '.join(word.split("'")))
    return ' '.join(sentence)


def tweet_text_preprocess(text):
    translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
    text=re.sub(r'@\w+',r'',re.sub(r'rt @\w+:',r'',re.sub(r'\\x[0-9A-Fa-f]+',r'', re.sub(r'\\n',r'',text)))) #,r'(?:RT @[\w_:]+) ',r''
    text=re.sub('(?:(?:https?|ftp):\/\/)?[\w/\-?=%]+\.[\w/\-?=%]+',r'',apos_words(text))
    text = ' '.join(re.findall("[\S]+", html.unescape(text))).replace("#"," ").replace("\n",'').replace("\t",'').replace("\r",'').replace("\f",'').replace("\v",'')
    return ' '.join(text.translate(translator).split()).lower()



def Zone_conversion(timestamp):
    
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('EDT')
    utc = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S')
    utc = utc.replace(tzinfo=from_zone)
    central = utc.astimezone(to_zone)
    return (str(central)[:-6])


def remove_emoji(doc):
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', doc)


global_stemmer = PorterStemmer()
wordnet_lemmatizer = WordNetLemmatizer()

def remove_name_and_https(corp):
    for doc in corp:
        yield  re.sub(r'\//t.','', re.sub(r'https.*\b\S',' ' , re.sub(r'\@\w+', ' ', doc)))
        
        
def remove_small_words(corp):
    for doc in corp:
        yield  re.sub(r'\brt|\bRT','',doc)


def read_corpus_basic(corp):
    for doc in corp:
        yield [x for x in gensim.utils.simple_preprocess(doc, deacc=True)]

def read_corpus_with_stemming_and_SW_removal(corp):
    for doc in corp:
        yield [global_stemmer.stem(x) for x in gensim.utils.simple_preprocess(doc, deacc=True)
                   if x.lower() not in STOPWORDS]
        
def read_corpus_with_lemmitizer_and_SW_removal(corp):
    for doc in corp:
        yield [wordnet_lemmatizer.lemmatize(x) for x in gensim.utils.simple_preprocess(doc, deacc=True)
                   if x.lower() not in STOPWORDS]
        

def get_senti_info(text,senti):
    pos_count = neg_count = 0
    if senti == "score":
        return round(sid.polarity_scores(text)['compound'],1)
    else:
        if senti == "pos":
            for w in text.split(" "):
                if sid.polarity_scores(w)['pos'] == 1:
                    pos_count+=1
        elif senti == "neg":
            for w in text.split(" "):
                if sid.polarity_scores(w)['neg'] == 1:
                    neg_count+=1
        else: pass
        return pos_count if senti == "pos" else neg_count        


In [3]:
global apos_dict
apos_dict = {
"aren't" : "are not",
"can't" : "can not",
"couldn't" : "could not",
"didn't" : "did not",
"doesn't" : "does not",
"don't" : "do not",
"hadn't" : "had not",
"hasn't" : "has not",
"haven't" : "have not",
"he'd" : "he would",
"he'll" : "he will",
"he's" : "he is",
"i'd" : "I would",
"i'll" : "I will",
"i'm" : "I am",
"I'd" : "I would",
"I'll" : "I will",
"I'm" : "I am",
"isn't" : "is not",
"it's" : "it is",
"it'll":"it will",
"i've" : "I have",
"let's" : "let us",
"mightn't" : "might not",
"mustn't" : "must not",
"shan't" : "shall not",
"she'd" : "she would",
"she'll" : "she will",
"she's" : "she is",
"shouldn't" : "should not",
"that's" : "that is",
"there's" : "there is",
"they'd" : "they would",
"they'll" : "they will",
"they're" : "they are",
"they've" : "they have",
"we'd" : "we would",
"we're" : "we are",
"weren't" : "were not",
"we've" : "we have",
"what'll" : "what will",
"what're" : "what are",
"what's" : "what is",
"what've" : "what have",
"where's" : "where is",
"who'd" : "who would",
"who'll" : "who will",
"who're" : "who are",
"who's" : "who is",
"who've" : "who have",
"won't" : "will not",
"wouldn't" : "would not",
"y'all": "you all",
"you'd" : "you would",
"you'll" : "you will",
"you're" : "you are",
"you've" : "you have",
"'re": " are",
"wasn't": "was not",
"we'll":"we will",
"didn't": "did not"
}

In [4]:
puncs=[]
for i in punctuation:
    puncs.append(str(i))

list1 = ['RT','rt', 'na', 'fx', 'th', 'http', 'lt', 'v', 'hd', 'pm', 'im', 'ga', 'amp', 'got', 'jf', 'ni'
         'db', 'way', 'ge', 'nv', 'ol', 'bd', 'ba', 'nf', 'se', 'dj', 'gt', 'georgia', 'hi', 'href', 'new']
List2 = ['#atlanta', u'#atlanta',  '#atlantas', u'#atlantas', '#ga ', '"', '#ga']
List2.extend(['this','that','the','might','have','been','from','ve',
                'but','they','will','has','having','had','how','went'
                'were','why','and','still','his','her','was','its','per','cent',
                'a','able','about','across','after','all','almost','also','am','among',
                'an','and','any','are','as','at','be','because','been','but','by','can',
                'cannot','could','dear','did','do','does','either','else','ever','every',
                'for','from','get','got','had','has','have','he','her','hers','him','his',
                'how','however','i','if','in','into','is','it','its','just','least','let',
                'like','likely','may','me','might','most','must','my','neither','nor','ll',
                'not','of','off','often','on','only','or','other','our','own','rather','said',
                'say','says','she','should','since','so','some','than','that','the','their','dez',
                'them','then','there','these','they','this','tis','to','too','twas','us','nufx','cqw',
                'wants','was','we','were','what','when','where','which','while','who','vykvyjcn',
                'whom','why','will','with','would','yet','you','your','ve','re','rt', 'retweet', '#fuckem', '#fuck',
                'fuck', 'ya', 'yall', 'yay', 'youre', 'youve', 'ass','factbox', 'com', '&lt', 'th',
                'retweeting', 'dick', 'fuckin', 'shit', 'via', 'fucking', 'shocker', 'wtf', 'hey', 'ooh', 'rt&amp', '&amp',
                '#retweet', 'retweet', 'goooooooooo', 'hellooo', 'gooo', 'fucks', 'fucka', 'bitch', 'wey', 'sooo', 
              'helloooooo', 'lol', 'smfh', 'bitch', 'atlanta', 'black', 'nigga', 'georgia', 'forward', 'today','tho','qjlgjezq'])




In [5]:
header= True

In [ ]:
#Hadoop Command to get data to local repository(Daily Files)
from datetime import datetime, date, timedelta

while True:
    try:
        a=datetime.now()
        
        yesterday = date.today() - timedelta(1)
        
        dates=str(a.year)+str(a.month).zfill(2)+str(a.day).zfill(2)
        #date1=str(a.year)+'-'+str(a.month).zfill(2)+'-'+str(a.day-1).zfill(2)
        
        yesterday_date=str(yesterday.strftime('%m-%d-%Y'))

        #print(yesterday_date)
        (ret, out, err)= run_cmd(['hadoop', 'fs', '-get', '/data/atl_sprint_2018/reddit_archive/reddit%sT0000.json'
                            %(dates), 'reddit_data/'])

        reddit_temp=[]
        Reddit_Data = pd.DataFrame()
        time.sleep(30) 
        with open('%sreddit%sT0000.json' %(path_to_json,dates), 'r') as f:
            for line in f:
                try:
                    reddit = json.loads(line)
                    reddit_temp.append(reddit)
                except:
                    continue 
        #Reddit_Data = pd.read_csv('%sreddit%sT0000.csv' %(path_to_json,dates), encoding='utf-8')
        Reddit_Data=Reddit_Data.from_records(reddit_temp)
        Reddit_Data['TimeZone_old'] = Reddit_Data[['Date', 'Time']].apply(lambda x: ' '.join(x), axis=1)
        Reddit_Data['TimeZone_new'] = [ Zone_conversion(i) for i in Reddit_Data.TimeZone_old]
        Reddit_Data['Date_new'] = [ datetime.strptime(str(i)[0:10],"%Y-%m-%d") for i in Reddit_Data.TimeZone_new]
        ##new code below
        #-------------------
        #Reddit_Data['Date_new'] = yesterday_date
        #--------------------------------
        Reddit_Data['Time_new'] = [ str(i)[11:] for i in Reddit_Data.TimeZone_new]
        Reddit_Data['CleanedText'] = [tweet_text_preprocess(str(i)) for i in Reddit_Data.Text]
        Reddit_Data['polarity'] = [round(sid.polarity_scores(i)['compound'],1) for i in Reddit_Data.CleanedText]
        #print(Reddit_Data)
        #d = a - timedelta(days=1)
        mode = 'w' if header else 'a'
        Reddit_Data[Reddit_Data.Date_new==yesterday_date].groupby(["Date_new"]).Text.count().reset_index(name="Volume").to_csv("Reddit_Output/Reddit_Daily_Volume.csv",sep=",",header=header,mode=mode,index=False)
        sent_temp=[]
        for i in range(0,len(Reddit_Data)):
            if Reddit_Data['polarity'][i]<0.0:
                sent_temp.append('Negative')
            elif Reddit_Data['polarity'][i]>0.0:
                sent_temp.append('Positive')
            else:
                sent_temp.append('Neutral')
        Reddit_Data['Sentiment']=sent_temp
        Reddit_Data[Reddit_Data.Date_new==yesterday_date].groupby(["Date_new","Sentiment"]).Text.count().reset_index(name="Volume").to_csv("Reddit_Output/Reddit_Overall_Sentiment.csv",sep=",",header=header,mode=mode,index=False)
        header=False
    
        temp=[]
        corpus=[]
        list_corp=list(Reddit_Data[Reddit_Data.Date_new==yesterday_date]['CleanedText'])
        for i in range(len(Reddit_Data[Reddit_Data.Date_new==yesterday_date]['CleanedText'])):
            temp=apos_words(remove_emoji(str(list_corp[i])))
            corpus.append(temp)

        corpus2 =  list(read_corpus_with_lemmitizer_and_SW_removal(
                    list( remove_small_words(
                        list(remove_name_and_https(
                            list(corpus)))))))

        #Selecting 2 letter words for exclusion    
        text_check=[x for doc in corpus2 for x in doc]
        two_letters = list(set([word for word in text_check if len(word) <= 2 and word not in stopwords.words('english')]))    
        stoplist = list1+puncs+two_letters

        text_token = [[x for x in doc if x not in stoplist] for doc in corpus2]
        flat_list = [item for sublist in text_token for item in sublist]
        word_list = nltk.FreqDist(flat_list)


        master_word_list = []
        for k, v in word_list.items():
            temp =[]
            temp.append(k)
            temp.append(v)
            master_word_list.append(temp)

        headers = ['Word','Freq']
        final_df = pd.DataFrame(master_word_list, columns=headers)

        final_df.to_csv("Reddit_Output/Reddit_WordCloud.csv",sep=",",header=True,mode='w')
        
###################Clearing the Home Folder after every 7 days######################       
        file_list=[]
        for the_file in os.listdir(path_to_json):
            file_list.append(the_file)
        try:
            if len(file_list)==7:
                for f in file_list:
                    os.remove(os.path.join(path_to_json,f))       
        except Exception as e:
            print(e)

        time.sleep(86400)
    except Exception as ex:
        print(ex)
        time.sleep(3600)
        continue
                    

Running system command: hadoop fs -get /data/atl_sprint_2018/reddit_archive/reddit20190208T0000.json reddit_data/
name 'header' is not defined
